In [4]:
'''
Visualization of the filters of a CNN, via gradient ascent in input space.
This script can run on CPU in a few minutes.
This script is meant for use only and is not discussed in great details as it is beyond the scope of the class.
'''
# using tf.2.1 in colab
# %tensorflow_version 2.x

# from __future__ import print_function

import numpy as np
import time
import tensorflow as tf
from tensorflow.keras.preprocessing.image import save_img
from tensorflow.keras import backend as K
# from google.colab import drive
# stopping eager execution mode to be allowed to use the gradient function
tf.compat.v1.disable_eager_execution()
# sanity check for tf version
print(tf.__version__)

###################################################################################################
# dimensions of the generated pictures for each filter.
###################################################################################################   
img_width = 150
img_height = 150

###################################################################################################
# the name of the layer we want to visualize 
###################################################################################################   
layer_name='conv2d_9'

###################################################################################################
# util function to convert a tensor into a valid image
###################################################################################################   
def deprocess_image(x):
    
    # normalize tensor: center on 0., ensure std is 0.1
    x -= x.mean()
    x /= (x.std() + K.epsilon())
    x *= 0.1

    # clip to [0, 1]
    x += 0.5
    x = np.clip(x, 0, 1)

    # convert to RGB array
    x *= 255
    if K.image_data_format() == 'channels_first':
        x = x.transpose((1, 2, 0))
    x = np.clip(x, 0, 255).astype('uint8')
    return x

###################################################################################################
# Load the model
###################################################################################################   
from tensorflow.keras.models import load_model
model=load_model('./dogs_cats_birds_model_case04_30epoch')

###################################################################################################
# This is the placeholder for the input images
###################################################################################################   
input_img = model.input

###################################################################################################
# Get the symbolic outputs of each "key" layer (we gave them unique names).
###################################################################################################   
layer_dict = dict([(layer.name, layer) for layer in model.layers[1:]])

###################################################################################################
# Utility function to normalize a tensor by its L2 norm
###################################################################################################   
def normalize(x): 
    return x / (K.sqrt(K.mean(K.square(x))) + K.epsilon())

kept_filters = []

###################################################################################################
# Scan through some number of filters...
###################################################################################################   

for filter_index in range(64):

    print('Processing filter %d' % filter_index)
    start_time = time.time()

    # we build a loss function that maximizes the activation
    # of the nth filter of the layer considered
    layer_output = layer_dict[layer_name].output
    if K.image_data_format() == 'channels_first':
        loss = K.mean(layer_output[:, filter_index, :, :])
    else:
        loss = K.mean(layer_output[:, :, :, filter_index])

    # we compute the gradient of the input picture wrt this loss
    grads = K.gradients(loss, input_img)[0]

    # normalization trick: we normalize the gradient
    grads = normalize(grads)

    # this function returns the loss and grads given the input picture
    iterate = K.function([input_img], [loss, grads])

    # step size for gradient ascent
    step = 1.

    # we start from a gray image with some random noise
    if K.image_data_format() == 'channels_first':
        input_img_data = np.random.random((1, 3, img_width, img_height))
    else:
        input_img_data = np.random.random((1, img_width, img_height, 3))
    input_img_data = (input_img_data - 0.5) * 20 + 128

    # we run gradient ascent for 20 steps
    for i in range(100):
        loss_value, grads_value = iterate([input_img_data])
        input_img_data += grads_value * step

        print('Current loss value:', loss_value)

    # decode the resulting input image
    if True:
        
    #if loss_value > 0
        img = deprocess_image(input_img_data[0])
        kept_filters.append((img, loss_value))
    end_time = time.time()
    print('Filter %d processed in %ds' % (filter_index, end_time - start_time))

###################################################################################################
# we will stich the best n^2 filters on a n x n grid.
###################################################################################################
n = 8

###################################################################################################
# the filters that have the highest loss are assumed to be more intuitive
# we will only keep the top n filters.
###################################################################################################   
kept_filters.sort(key=lambda x: x[1], reverse=True)
kept_filters = kept_filters[:n * n]

###################################################################################################
# build a black picture with enough space for
# our n x n filters of size 128 x 128, with a 5px margin in between
###################################################################################################   
margin = 5
width = n * img_width + (n - 1) * margin
height = n * img_height + (n - 1) * margin
stitched_filters = np.zeros((width, height, 3))

###################################################################################################
# fill the picture with our saved filters 
###################################################################################################   
for i in range(n):
    for j in range(n):
        img, loss = kept_filters[i * n + j]
        stitched_filters[(img_width + margin) * i: (img_width + margin) * i + img_width,
                         (img_height + margin) * j: (img_height + margin) * j + img_height, :] = img

###################################################################################################
# save the result to drive
###################################################################################################   
save_img('./dogs_cats_birds_model_case04_30epoch_%dx%d.png' % (n, n), stitched_filters)
print('saved')

2.1.0
Processing filter 0
Current loss value: -20.307362
Current loss value: -11.139647
Current loss value: -1.6646224
Current loss value: 2.8847733
Current loss value: 7.4217362
Current loss value: 10.657519
Current loss value: 13.080965
Current loss value: 16.607601
Current loss value: 19.805061
Current loss value: 24.129547
Current loss value: 30.702225
Current loss value: 36.67931
Current loss value: 42.617424
Current loss value: 49.712757
Current loss value: 56.783978
Current loss value: 64.09324
Current loss value: 71.73967
Current loss value: 79.52046
Current loss value: 88.886765
Current loss value: 97.83194
Current loss value: 106.57781
Current loss value: 116.83882
Current loss value: 128.01208
Current loss value: 138.99585
Current loss value: 149.5568
Current loss value: 162.05098
Current loss value: 172.9121
Current loss value: 185.1398
Current loss value: 198.37231
Current loss value: 210.5058
Current loss value: 223.50496
Current loss value: 238.39484
Current loss value: 

Current loss value: -1.9698924
Current loss value: 5.4099655
Current loss value: 9.731712
Current loss value: 14.105043
Current loss value: 16.416832
Current loss value: 18.96265
Current loss value: 22.028183
Current loss value: 29.37713
Current loss value: 42.361908
Current loss value: 72.477936
Current loss value: 112.7562
Current loss value: 164.80098
Current loss value: 226.09225
Current loss value: 290.44043
Current loss value: 359.4713
Current loss value: 430.51263
Current loss value: 503.26892
Current loss value: 572.5602
Current loss value: 649.35724
Current loss value: 726.05994
Current loss value: 804.5864
Current loss value: 883.5449
Current loss value: 965.7117
Current loss value: 1049.5428
Current loss value: 1133.8933
Current loss value: 1218.9818
Current loss value: 1302.3888
Current loss value: 1388.9336
Current loss value: 1474.7468
Current loss value: 1562.2192
Current loss value: 1648.9338
Current loss value: 1739.8835
Current loss value: 1827.2327
Current loss value

Current loss value: 2763.5522
Current loss value: 2802.107
Current loss value: 2842.2322
Current loss value: 2877.9824
Current loss value: 2917.7627
Current loss value: 2956.5269
Current loss value: 2993.1162
Current loss value: 3032.7317
Current loss value: 3069.1892
Current loss value: 3107.9893
Current loss value: 3145.2305
Current loss value: 3182.932
Current loss value: 3222.135
Current loss value: 3259.07
Current loss value: 3295.4807
Current loss value: 3331.8179
Current loss value: 3369.5605
Current loss value: 3406.5837
Current loss value: 3443.6245
Current loss value: 3478.5132
Current loss value: 3515.5996
Current loss value: 3551.9595
Filter 5 processed in 1s
Processing filter 6
Current loss value: -26.966293
Current loss value: -13.983999
Current loss value: -3.4212246
Current loss value: -0.04256974
Current loss value: -0.05515498
Current loss value: 0.81823295
Current loss value: 0.18584791
Current loss value: 1.4736326
Current loss value: 1.809523
Current loss value: 2.

Current loss value: 93.514114
Current loss value: 96.12187
Current loss value: 97.9889
Current loss value: 100.2799
Current loss value: 102.82129
Current loss value: 104.695435
Current loss value: 106.52191
Current loss value: 109.10689
Current loss value: 111.055756
Current loss value: 113.1999
Current loss value: 115.91519
Current loss value: 118.26699
Current loss value: 120.866196
Current loss value: 122.58237
Current loss value: 125.20736
Current loss value: 127.540985
Current loss value: 129.14671
Current loss value: 132.4725
Current loss value: 135.33868
Current loss value: 137.70401
Current loss value: 140.09166
Current loss value: 142.57184
Current loss value: 145.66986
Current loss value: 147.29755
Current loss value: 150.73334
Filter 8 processed in 1s
Processing filter 9
Current loss value: 8.854695
Current loss value: 14.851499
Current loss value: 18.10522
Current loss value: 21.314598
Current loss value: 25.09625
Current loss value: 29.839218
Current loss value: 35.11714
C

Current loss value: 72.892105
Current loss value: 75.43797
Current loss value: 77.85051
Current loss value: 80.0883
Current loss value: 82.68071
Current loss value: 84.11296
Current loss value: 86.47633
Current loss value: 88.672386
Current loss value: 90.40883
Current loss value: 93.65083
Current loss value: 93.74123
Current loss value: 98.09329
Current loss value: 98.09206
Current loss value: 102.327644
Current loss value: 102.44149
Current loss value: 105.64168
Current loss value: 109.56267
Current loss value: 110.72203
Current loss value: 114.11514
Current loss value: 115.083565
Current loss value: 117.85113
Current loss value: 120.486115
Current loss value: 122.6758
Current loss value: 124.230835
Current loss value: 126.88068
Filter 11 processed in 1s
Processing filter 12
Current loss value: 11.897201
Current loss value: 16.315056
Current loss value: 18.060995
Current loss value: 21.95522
Current loss value: 25.914413
Current loss value: 29.413391
Current loss value: 33.914436
Cur

...

Current loss value: 111.51245
Current loss value: 111.84289
Current loss value: 114.37653
Current loss value: 115.869
Current loss value: 117.14273
Current loss value: 118.17404
Current loss value: 120.695045
Current loss value: 122.17955
Current loss value: 124.702515
Current loss value: 125.973465
Current loss value: 126.9869
Current loss value: 128.89255
Current loss value: 130.62619
Current loss value: 132.93268
Current loss value: 133.03876
Current loss value: 135.01639
Current loss value: 136.78801
Current loss value: 139.16844
Current loss value: 139.9708
Current loss value: 142.15494
Current loss value: 143.58228
Filter 50 processed in 1s
Processing filter 51
Current loss value: -14.139279
Current loss value: -7.2479343
Current loss value: -2.6749735
Current loss value: -0.2633459
Current loss value: 0.95314175
Current loss value: 3.1393769
Current loss value: 5.003444
Current loss value: 8.019751
Current loss value: 9.946662
Current loss value: 10.28409
Current loss value: 11.

Current loss value: 1767.1932
Current loss value: 1790.9031
Current loss value: 1811.2404
Current loss value: 1836.4799
Current loss value: 1859.2765
Current loss value: 1883.154
Current loss value: 1905.9215
Current loss value: 1928.5571
Current loss value: 1953.3158
Current loss value: 1975.6299
Current loss value: 2002.1902
Current loss value: 2023.5089
Current loss value: 2050.2227
Current loss value: 2073.0137
Current loss value: 2100.92
Current loss value: 2121.308
Current loss value: 2153.6714
Current loss value: 2178.9463
Current loss value: 2198.945
Current loss value: 2221.9956
Current loss value: 2250.3423
Current loss value: 2270.5688
Current loss value: 2296.652
Current loss value: 2321.3274
Current loss value: 2348.881
Current loss value: 2370.7854
Current loss value: 2394.7817
Filter 53 processed in 1s
Processing filter 54
Current loss value: 3.6765823
Current loss value: 5.7115126
Current loss value: 7.414177
Current loss value: 9.196199
Current loss value: 11.441283
Cu

Current loss value: 1280.7585
Current loss value: 1316.117
Current loss value: 1350.3779
Current loss value: 1388.912
Current loss value: 1429.7805
Current loss value: 1472.7251
Current loss value: 1509.6244
Current loss value: 1554.6145
Current loss value: 1595.6663
Current loss value: 1638.1301
Current loss value: 1682.5625
Current loss value: 1728.0117
Current loss value: 1775.3961
Current loss value: 1824.9485
Current loss value: 1873.5115
Current loss value: 1925.0845
Current loss value: 1975.7476
Current loss value: 2028.7222
Current loss value: 2082.5186
Current loss value: 2136.7703
Current loss value: 2196.911
Current loss value: 2257.768
Current loss value: 2322.3105
Current loss value: 2389.6719
Filter 56 processed in 1s
Processing filter 57
Current loss value: -32.611465
Current loss value: -19.872707
Current loss value: -5.793867
Current loss value: -3.8479104
Current loss value: -2.716305
Current loss value: -2.333796
Current loss value: -0.96933347
Current loss value: -1

Current loss value: 2879.3135
Current loss value: 2925.624
Current loss value: 2976.1074
Current loss value: 3021.0994
Current loss value: 3070.4626
Current loss value: 3117.0542
Current loss value: 3165.769
Current loss value: 3211.5674
Current loss value: 3259.7217
Current loss value: 3305.3867
Current loss value: 3351.727
Current loss value: 3399.7886
Current loss value: 3445.2837
Current loss value: 3489.613
Current loss value: 3536.8494
Current loss value: 3582.8337
Current loss value: 3626.476
Current loss value: 3668.7427
Current loss value: 3719.5488
Current loss value: 3765.4675
Current loss value: 3809.4644
Current loss value: 3858.749
Current loss value: 3906.1587
Current loss value: 3951.9153
Current loss value: 3998.4326
Current loss value: 4046.3562
Current loss value: 4093.0679
Filter 59 processed in 1s
Processing filter 60
Current loss value: 6.626912
Current loss value: 8.583635
Current loss value: 10.150328
Current loss value: 11.811785
Current loss value: 13.368433
C

Current loss value: 70.17081
Current loss value: 72.453476
Current loss value: 74.89603
Current loss value: 76.66072
Current loss value: 78.553215
Current loss value: 81.1346
Current loss value: 82.99248
Current loss value: 85.600555
Current loss value: 87.324905
Current loss value: 89.662575
Current loss value: 90.35465
Current loss value: 92.95768
Current loss value: 94.7669
Current loss value: 97.51808
Current loss value: 98.75065
Current loss value: 100.31338
Current loss value: 102.82463
Current loss value: 104.1011
Current loss value: 106.286194
Current loss value: 108.22539
Current loss value: 110.62959
Current loss value: 111.87468
Current loss value: 114.268295
Current loss value: 116.54163
Current loss value: 118.19295
Current loss value: 121.338974
Current loss value: 123.365524
Current loss value: 125.30936
Filter 62 processed in 1s
Processing filter 63
Current loss value: -20.466183
Current loss value: -10.97224
Current loss value: -3.2648165
Current loss value: -1.589431
